In [9]:
import tensorflow as tf
import pandas as pd
tf.__version__

'2.13.0'

In [10]:
# Input data
df = pd.read_csv('../input_data/CSVs/superconductivity.csv')
# Drop columns with null entries
df.dropna(inplace=True)
df

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.00
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.00
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.00
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.00
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21258,4,106.957877,53.095769,82.515384,43.135565,1.177145,1.254119,146.88130,15.504479,65.764081,...,3.555556,3.223710,3.519911,1.377820,0.913658,1,2.168889,0.433013,0.496904,2.44
21259,5,92.266740,49.021367,64.812662,32.867748,1.323287,1.571630,188.38390,7.353333,69.232655,...,2.047619,2.168944,2.038991,1.594167,1.337246,1,0.904762,0.400000,0.212959,122.10
21260,2,99.663190,95.609104,99.433882,95.464320,0.690847,0.530198,13.51362,53.041104,6.756810,...,4.800000,4.472136,4.781762,0.686962,0.450561,1,3.200000,0.500000,0.400000,1.98
21261,2,99.663190,97.095602,99.433882,96.901083,0.690847,0.640883,13.51362,31.115202,6.756810,...,4.690000,4.472136,4.665819,0.686962,0.577601,1,2.210000,0.500000,0.462493,1.84


In [11]:
# Add a boolean to the entries based on if they are above or below 77.36 degrees critical temp - 2 categories to select from
target = df['critical_temp'].apply(lambda x: 1 if x > 77.36 else 0)

features = df.drop('critical_temp', axis=1)

In [12]:
from sklearn.preprocessing import StandardScaler
# Standard scaling around 0
scaler = StandardScaler()
feature_array = scaler.fit_transform(features)
features = pd.DataFrame(feature_array, columns=features.columns)

In [13]:
model = tf.keras.Sequential()

# Input layer matching feature set shape
model.add(tf.keras.layers.InputLayer(input_shape=features.shape[1], name='Input_layer'))
# Dense hidden layers for processing
model.add(tf.keras.layers.Dense(32, name='Hidden_layer_1'))
model.add(tf.keras.layers.Dense(16, name='Hidden_layer_2'))
model.add(tf.keras.layers.Dense(8, name='Hidden_layer_3'))
# Final output with sigmoid activation function
model.add(tf.keras.layers.Dense(1, name='Output_layer', activation='sigmoid'))

In [14]:
# Use RMSprop optimiser and binary cross entropy loss function (as there are only 2 categories to choose from)
model.compile(tf.optimizers.RMSprop(0.0001), loss='binary_crossentropy', metrics=['accuracy'])
# Logging for tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [15]:
# Train AI on 80% of the data over 50 epochs
model.fit(x=features.to_numpy(), y=target.to_numpy(), epochs=50, callbacks=[tensorboard_callback], validation_split=0)

Epoch 1/50
665/665 [==============================] - 2s 2ms/step - loss: 0.4895 - accuracy: 0.7508
Epoch 2/50
665/665 [==============================] - 1s 2ms/step - loss: 0.3045 - accuracy: 0.8491
Epoch 3/50
665/665 [==============================] - 1s 2ms/step - loss: 0.2680 - accuracy: 0.8569
Epoch 4/50
665/665 [==============================] - 1s 2ms/step - loss: 0.2568 - accuracy: 0.8622
Epoch 5/50
665/665 [==============================] - 1s 2ms/step - loss: 0.2509 - accuracy: 0.8642
Epoch 6/50
665/665 [==============================] - 1s 2ms/step - loss: 0.2465 - accuracy: 0.8680
Epoch 7/50
665/665 [==============================] - 1s 2ms/step - loss: 0.2437 - accuracy: 0.8693
Epoch 8/50
665/665 [==============================] - 1s 2ms/step - loss: 0.2412 - accuracy: 0.8714
Epoch 9/50
665/665 [==============================] - 1s 2ms/step - loss: 0.2391 - accuracy: 0.8704
Epoch 10/50
665/665 [==============================] - 1s 2ms/step - loss: 0.2379 - accuracy: 0.8742

In [16]:
# Test on the remaining 20%
loss, accuracy = model.evaluate(features.to_numpy(), target.to_numpy())

print(f'loss: {loss}, accuracy: {accuracy}')

665/665 [==============================] - 1s 1ms/step - loss: 0.2217 - accuracy: 0.8928
loss: 0.22171299159526825, accuracy: 0.8928185105323792
